# Part 1

Diagnostics indicate that the local grid computing cluster has been contaminated with the Sporifica Virus. The grid computing cluster is a seemingly-infinite two-dimensional grid of compute nodes. Each node is either clean or infected by the virus.

To prevent overloading the nodes (which would render them useless to the virus) or detection by system administrators, exactly one virus carrier moves through the network, infecting or cleaning nodes as it moves. The virus carrier is always located on a single node in the network (the current node) and keeps track of the direction it is facing.

To avoid detection, the virus carrier works in bursts; in each burst, it wakes up, does some work, and goes back to sleep. The following steps are all executed in order one time each burst:

- If the current node is infected, it turns to its right. Otherwise, it turns to its left. (Turning is done in-place; the current node does not change.)
- If the current node is clean, it becomes infected. Otherwise, it becomes cleaned. (This is done after the node is considered for the purposes of changing direction.)
- The virus carrier moves forward one node in the direction it is facing.

Diagnostics have also provided a map of the node infection status (your puzzle input). Clean nodes are shown as `.`; infected nodes are shown as `#`. This map only shows the center of the grid; there are many more nodes beyond those shown, but none of them are currently infected.

The virus carrier begins in the middle of the map facing up.

For example, suppose you are given a map like this:

```
..#
#..
...
```

Then, the middle of the infinite grid looks like this, with the virus carrier's position marked with [ ]:

```
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . . #[.]. . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
```

The virus carrier is on a clean node, so it turns left, infects the node, and moves left:

```
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . .[#]# . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
```

The virus carrier is on an infected node, so it turns right, cleans the node, and moves up:

```
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . .[.]. # . . .
. . . . # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
```

Four times in a row, the virus carrier finds a clean, infects it, turns left, and moves forward, ending in the same place and still facing up:

```
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . #[#]. # . . .
. . # # # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
```

Now on the same node as before, it sees an infection, which causes it to turn right, clean the node, and move forward:

```
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . # .[.]# . . .
. . # # # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
```

After the above actions, a total of 7 bursts of activity had taken place. Of them, 5 bursts of activity caused an infection.

After a total of 70, the grid looks like this, with the virus carrier facing up:

```
. . . . . # # . .
. . . . # . . # .
. . . # . . . . #
. . # . #[.]. . #
. . # . # . . # .
. . . . . # # . .
. . . . . . . . .
. . . . . . . . .
```

By this time, 41 bursts of activity caused an infection (though most of those nodes have since been cleaned).

After a total of 10000 bursts of activity, 5587 bursts will have caused an infection.

Given your actual map, after 10000 bursts of activity, how many bursts cause a node to become infected? (Do not count nodes that begin infected.)

In [36]:
from collections import defaultdict
from io import StringIO
from itertools import chain, product
from textwrap import dedent

In [16]:
def load_grid(rows):
    grid = defaultdict(lambda: '.')
    grid_size = None
    for row_n, row in enumerate(rows):
        row = row.strip()
        if grid_size is None:
            grid_size = len(row)
            middle = grid_size // 2
        grid.update(((col_n - middle, middle - row_n), c) for col_n, c in enumerate(row))
    return grid

In [41]:
def _test_input():
    return load_grid(StringIO(dedent("""\
        ..#
        #..
        ...
        """)))
TEST_INPUT = _test_input()

In [19]:
def _puzzle_input():
    with open('./inputs/day22/input.txt') as f:
        return load_grid(f)
PUZZLE_INPUT = _puzzle_input()

In [44]:
def draw_grid(grid: dict):
    grid_size = max(abs(x) for x in chain.from_iterable(grid.keys()))
    grid_str = '\n'.join(
        ''.join(grid[(col_n, row_n)] for col_n in range(-grid_size, grid_size + 1))
        for row_n in range(grid_size, -grid_size - 1, -1))
    print(grid_str)

In [45]:
draw_grid(TEST_INPUT)

..#
#..
...


In [1]:
moves = {
    'n': (0, 1),
    's': (0, -1),
    'e': (1, 0),
    'w': (-1, 0)
}

turns = {
    'n': {'left': 'w', 'right': 'e'},
    's': {'left': 'e', 'right': 'w'},
    'e': {'left': 'n', 'right': 's'},
    'w': {'left': 's', 'right': 'n'}
}

In [20]:
def add(p1, p2):
    return (p1[0] + p2[0], p1[1] + p2[1])

In [26]:
def burst(grid: dict, pos: tuple, dir_: str):
    if grid[pos] == '.':
        new_dir = turns[dir_]['left']
        grid[pos] = '#'
        new_pos = add(pos, moves[new_dir])
        infected = True
    else:
        new_dir = turns[dir_]['right']
        grid[pos] = '.'
        new_pos = add(pos, moves[new_dir])
        infected = False
    return grid, new_pos, new_dir, infected

In [29]:
def do_iterations(grid: dict, iterations: int) -> int:
    grid = grid.copy()
    pos = (0, 0)
    dir_ = 'n'
    infected_count = 0
    
    for _ in range(iterations):
        _, pos, dir_, infected = burst(grid, pos, dir_)
        infected_count += infected
    
    return infected_count, grid, pos, dir_

In [47]:
result, *rest = do_iterations(TEST_INPUT, 7)
assert result == 5, result
result, *rest = do_iterations(TEST_INPUT, 70)
assert result == 41, result
result, *rest = do_iterations(TEST_INPUT, 10000)
assert result == 5587, result

In [48]:
result, *rest = do_iterations(PUZZLE_INPUT, 10000)
result

5433